In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import StandardScaler
from scipy import stats as s
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
import sklearn.cross_validation as crv
from sklearn.metrics import accuracy_score

/usr/local/opt/pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def col_name():
    p0 = ['x','y','z']
    marker =[]
    p2 =[]
    for a in range(1,44):       #Mocapマーカー数:43個
        p1='m'+ str(a)
        for b in p0:
            p2 = p1 + b
            marker.append(p2)
    return marker

In [3]:
def read_mocap(file):
    data_mocap = pd.read_csv(file,sep=' ',header=None)
    #座標値列の列名変更
    data_mocap=data_mocap.rename(columns={data_mocap.columns[a]:col_name()[a] for a in range(129)} )
    #座標値列以外の列名変更
    data_mocap=data_mocap.rename(columns={129:'FrameNumber',130:'TimeStamp'})
    return data_mocap

In [4]:
#fix NaN
def fix_NaN(data):
    for  c  in  data.columns:
        col = data[c]  #'X'列から1列ずつ見る
        for r in range(len(data)):  #1行ずつ見る
            if col[r] == 0.0:       #0値が見つかれば
                data.iloc[r][c] = 'NaN' #'NaN'を代入
    mc_clean = data.interpolate()
    mc_clean = mc_clean.fillna(0)
    return mc_clean

def obtain_statistical_feature(window):
    #features = np.array(np.array(np.var(window, axis=0)))
    #features=np.array(np.array(np.mean(window-np.mean(window), axis=0)))
    features=np.array( np.array(np.mean(window-np.median(window), axis=0))) 
    features=np.append(features, np.array(np.std(window, axis=0))) 
    features=np.append(features, np.array(np.var(window, axis=0)))
    features=np.append(features, np.array(s.skew(window, axis=0)))
    features=np.append(features, np.array(s.kurtosis(window, axis=0)))
    features=np.append(features, np.array(get_tw_variance(window)))
    return features.reshape(1,len(features))


def get_tw_variance(window):
    total_cols = window.shape[1]
    tw_var = []
    for i in range (total_cols):
        column = window.iloc[:,i]
        tw_var.append(get_tw_col_var(column))
    return tw_var


def get_tw_col_var(column):
    mean = np.mean(column)
    total = len(column)
    
    v_sum = 0
    for i in range(total):
        w = np.exp(-0.5*(total-i))
        #print("el peso", w, "indice", i, "total", total)
        v = w*np.square(column.iloc[i]-mean)
        v_sum += v
    return v_sum

In [5]:
#速度加速度を求める関数を定義
def cal_va(data):
    diffs = pd.DataFrame()
    for i in range(len(data)-1):
        diff = data.iloc[i+1]-data.iloc[i]
        diffs = diffs.append(diff,ignore_index=True)
        
    vs = pd.DataFrame()
    for i in range(len(diffs)):
        v = diffs.iloc[i][1:]/diffs.iloc[i]["TimeStamp"]
        vs = vs.append(v,ignore_index=True)
    return vs

# Mocapのみで行動認識

In [70]:
def only_mocap_nods(*markers):
    x = np.empty([0,3*6*len(markers)])
    y = np.empty([0,])
    #Read all mocap data
    subjects =  ["01","02","03","04","05","06","07","08","09","10","11","12"]
    actions = ["01","02","03","04","05","06","07","08","09","10","11"]
    #subjects = ["01"]
    #actions = ["01"]
    for subject in subjects:
        for action in actions:
            for record in range(1,6):
                try:
                    data = read_mocap('/Users/takeshin/ws/BerkeleyMHAD/Mocap/OpticalData/moc_s'+str(subject)+'_a'+str(action)+'_r0'+str(record)+'.txt')
                    print('moc_s'+str(subject)+'_a'+str(action)+'_r0'+str(record)+'.txt')
                    columns = [['m'+str(i)+'x','m'+str(i)+'y','m'+str(i)+'z'] for i in markers]
                    cols = np.array(columns)
                    cols = list(np.ravel(cols))
                    cols.append("TimeStamp")
                    data = data[cols]
                    
                    data = fix_NaN(data)
                    
                    i = 0
                    
                    sf = obtain_statistical_feature(data.loc[:, data.columns != 'TimeStamp'])
                    if np.isnan(sf).any() :
                        print('have nan') 
                    x =np.append(x,sf, axis=0)
                    y = np.append(y,[action], axis=0)
                
                
                except FileNotFoundError:
                    pass
                    
    return x, y

In [72]:
x,y=only_mocap_nods(30)

moc_s01_a01_r01.txt
moc_s01_a01_r02.txt
moc_s01_a01_r03.txt
moc_s01_a01_r04.txt
moc_s01_a01_r05.txt
moc_s01_a02_r01.txt
moc_s01_a02_r02.txt
moc_s01_a02_r03.txt
moc_s01_a02_r04.txt
moc_s01_a02_r05.txt
moc_s01_a03_r01.txt
moc_s01_a03_r02.txt
moc_s01_a03_r03.txt
moc_s01_a03_r04.txt
moc_s01_a03_r05.txt
moc_s01_a04_r01.txt
moc_s01_a04_r02.txt
moc_s01_a04_r03.txt
moc_s01_a04_r04.txt
moc_s01_a04_r05.txt
moc_s01_a05_r01.txt
moc_s01_a05_r02.txt
moc_s01_a05_r03.txt
moc_s01_a05_r04.txt
moc_s01_a05_r05.txt
moc_s01_a06_r01.txt
moc_s01_a06_r02.txt
moc_s01_a06_r03.txt
moc_s01_a06_r04.txt
moc_s01_a06_r05.txt
moc_s01_a07_r01.txt
moc_s01_a07_r02.txt
moc_s01_a07_r03.txt
moc_s01_a07_r04.txt
moc_s01_a07_r05.txt
moc_s01_a08_r01.txt
moc_s01_a08_r02.txt
moc_s01_a08_r03.txt
moc_s01_a08_r04.txt
moc_s01_a08_r05.txt
moc_s01_a09_r01.txt
moc_s01_a09_r02.txt
moc_s01_a09_r03.txt
moc_s01_a09_r04.txt
moc_s01_a09_r05.txt
moc_s01_a10_r01.txt
moc_s01_a10_r02.txt
moc_s01_a10_r03.txt
moc_s01_a10_r04.txt
moc_s01_a10_r05.txt


moc_s08_a06_r02.txt
moc_s08_a06_r03.txt
moc_s08_a06_r04.txt
moc_s08_a06_r05.txt
moc_s08_a07_r01.txt
moc_s08_a07_r02.txt
moc_s08_a07_r03.txt
moc_s08_a07_r04.txt
moc_s08_a07_r05.txt
moc_s08_a08_r01.txt
moc_s08_a08_r02.txt
moc_s08_a08_r03.txt
moc_s08_a08_r04.txt
moc_s08_a08_r05.txt
moc_s08_a09_r01.txt
moc_s08_a09_r02.txt
moc_s08_a09_r03.txt
moc_s08_a09_r04.txt
moc_s08_a09_r05.txt
moc_s08_a10_r01.txt
moc_s08_a10_r02.txt
moc_s08_a10_r03.txt
moc_s08_a10_r04.txt
moc_s08_a10_r05.txt
moc_s08_a11_r01.txt
moc_s08_a11_r02.txt
moc_s08_a11_r03.txt
moc_s08_a11_r04.txt
moc_s08_a11_r05.txt
moc_s09_a01_r01.txt
moc_s09_a01_r02.txt
moc_s09_a01_r03.txt
moc_s09_a01_r04.txt
moc_s09_a01_r05.txt
moc_s09_a02_r01.txt
moc_s09_a02_r02.txt
moc_s09_a02_r03.txt
moc_s09_a02_r04.txt
moc_s09_a02_r05.txt
moc_s09_a03_r01.txt
moc_s09_a03_r02.txt
moc_s09_a03_r03.txt
moc_s09_a03_r04.txt
moc_s09_a03_r05.txt
moc_s09_a04_r01.txt
moc_s09_a04_r02.txt
moc_s09_a04_r03.txt
moc_s09_a04_r04.txt
moc_s09_a04_r05.txt
moc_s09_a05_r01.txt


In [73]:
ml_ar(x,y)

0.7626262626262627
             precision    recall  f1-score   support

         01       1.00      0.68      0.81        22
         02       0.70      0.89      0.78        18
         03       0.95      0.95      0.95        20
         04       0.73      0.61      0.67        18
         05       0.60      0.60      0.60        20
         06       0.76      0.62      0.68        21
         07       0.61      0.73      0.67        15
         08       0.57      0.80      0.67        15
         09       0.71      0.92      0.80        13
         10       1.00      0.84      0.91        19
         11       0.88      0.82      0.85        17

avg / total       0.79      0.76      0.76       198

[[15  5  1  0  0  0  0  1  0  0  0]
 [ 0 16  0  0  0  1  0  1  0  0  0]
 [ 0  0 19  0  0  0  0  1  0  0  0]
 [ 0  1  0 11  0  1  0  5  0  0  0]
 [ 0  0  0  1 12  1  6  0  0  0  0]
 [ 0  0  0  1  5 13  1  1  0  0  0]
 [ 0  0  0  0  3  1 11  0  0  0  0]
 [ 0  1  0  2  0  0  0 12  0  0  0]
 

In [89]:
def only_mocap(sampling_rate,*markers):
    x = np.empty([0,3*6*len(markers)])
    y = np.empty([0,])
    #Read all mocap data
    subjects =  ["01","02","03","04","05","06","07","08","09","10","11","12"]
    actions = ["01","02","03","04","05","06","07","08","09","10","11"]

    for subject in subjects:
        for action in actions:
            for record in range(1,6):
                try:
                    data = read_mocap('/Users/takeshin/ws/BerkeleyMHAD/Mocap/OpticalData/moc_s'+str(subject)+'_a'+str(action)+'_r0'+str(record)+'.txt')
                    print('moc_s'+str(subject)+'_a'+str(action)+'_r0'+str(record)+'.txt')
                    columns = [['m'+str(i)+'x','m'+str(i)+'y','m'+str(i)+'z'] for i in markers]
                    cols = np.array(columns)
                    cols = list(np.ravel(cols))
                    cols.append("TimeStamp")
                    data = data[cols]
                    
                    data = fix_NaN(data)
                    i = 0
                    mc_ds = pd.DataFrame()
                    #処理
                    while  i < (len(data)//(480//sampling_rate))*(480//sampling_rate):
                        mc_ds = mc_ds.append(data.iloc[i : i+(480//sampling_rate),:].mean(),ignore_index=True)
                        i+=(480//sampling_rate)
                    if i < len(data):
                        mc_ds = mc_ds.append(data.iloc[i :,:].mean(),ignore_index=True)
                    sf = obtain_statistical_feature(mc_ds.loc[:, mc_ds.columns != 'TimeStamp'])
                    #if np.isnan(sf).any() :
                       # print("nan -----")
                        #print(np.isnan(mc_ds ).any()) 

                    x =np.append(x,sf, axis=0)
                    y = np.append(y,[action], axis=0)
                
                
                except FileNotFoundError:
                    pass
                    
    return x, y

In [90]:
x,y = only_mocap(30,30)

moc_s01_a01_r01.txt
moc_s01_a01_r02.txt
moc_s01_a01_r03.txt
moc_s01_a01_r04.txt
moc_s01_a01_r05.txt
moc_s01_a02_r01.txt
moc_s01_a02_r02.txt
moc_s01_a02_r03.txt
moc_s01_a02_r04.txt
moc_s01_a02_r05.txt
moc_s01_a03_r01.txt
moc_s01_a03_r02.txt
moc_s01_a03_r03.txt
moc_s01_a03_r04.txt
moc_s01_a03_r05.txt
moc_s01_a04_r01.txt
moc_s01_a04_r02.txt
moc_s01_a04_r03.txt
moc_s01_a04_r04.txt
moc_s01_a04_r05.txt
moc_s01_a05_r01.txt
moc_s01_a05_r02.txt
moc_s01_a05_r03.txt
moc_s01_a05_r04.txt
moc_s01_a05_r05.txt
moc_s01_a06_r01.txt
moc_s01_a06_r02.txt
moc_s01_a06_r03.txt
moc_s01_a06_r04.txt
moc_s01_a06_r05.txt
moc_s01_a07_r01.txt
moc_s01_a07_r02.txt
moc_s01_a07_r03.txt
moc_s01_a07_r04.txt
moc_s01_a07_r05.txt
moc_s01_a08_r01.txt
moc_s01_a08_r02.txt
moc_s01_a08_r03.txt
moc_s01_a08_r04.txt
moc_s01_a08_r05.txt
moc_s01_a09_r01.txt
moc_s01_a09_r02.txt
moc_s01_a09_r03.txt
moc_s01_a09_r04.txt
moc_s01_a09_r05.txt
moc_s01_a10_r01.txt
moc_s01_a10_r02.txt
moc_s01_a10_r03.txt
moc_s01_a10_r04.txt
moc_s01_a10_r05.txt


moc_s08_a06_r02.txt
moc_s08_a06_r03.txt
moc_s08_a06_r04.txt
moc_s08_a06_r05.txt
moc_s08_a07_r01.txt
moc_s08_a07_r02.txt
moc_s08_a07_r03.txt
moc_s08_a07_r04.txt
moc_s08_a07_r05.txt
moc_s08_a08_r01.txt
moc_s08_a08_r02.txt
moc_s08_a08_r03.txt
moc_s08_a08_r04.txt
moc_s08_a08_r05.txt
moc_s08_a09_r01.txt
moc_s08_a09_r02.txt
moc_s08_a09_r03.txt
moc_s08_a09_r04.txt
moc_s08_a09_r05.txt
moc_s08_a10_r01.txt
moc_s08_a10_r02.txt
moc_s08_a10_r03.txt
moc_s08_a10_r04.txt
moc_s08_a10_r05.txt
moc_s08_a11_r01.txt
moc_s08_a11_r02.txt
moc_s08_a11_r03.txt
moc_s08_a11_r04.txt
moc_s08_a11_r05.txt
moc_s09_a01_r01.txt
moc_s09_a01_r02.txt
moc_s09_a01_r03.txt
moc_s09_a01_r04.txt
moc_s09_a01_r05.txt
moc_s09_a02_r01.txt
moc_s09_a02_r02.txt
moc_s09_a02_r03.txt
moc_s09_a02_r04.txt
moc_s09_a02_r05.txt
moc_s09_a03_r01.txt
moc_s09_a03_r02.txt
moc_s09_a03_r03.txt
moc_s09_a03_r04.txt
moc_s09_a03_r05.txt
moc_s09_a04_r01.txt
moc_s09_a04_r02.txt
moc_s09_a04_r03.txt
moc_s09_a04_r04.txt
moc_s09_a04_r05.txt
moc_s09_a05_r01.txt


In [91]:
np.where(np.isnan(x))

(array([], dtype=int64), array([], dtype=int64))

In [92]:
ml_ar(x,y)

0.7424242424242424
             precision    recall  f1-score   support

         01       1.00      0.64      0.78        22
         02       0.65      0.94      0.77        18
         03       1.00      1.00      1.00        20
         04       0.79      0.61      0.69        18
         05       0.50      0.45      0.47        20
         06       0.67      0.57      0.62        21
         07       0.50      0.73      0.59        15
         08       0.61      0.73      0.67        15
         09       0.80      0.92      0.86        13
         10       1.00      0.74      0.85        19
         11       0.84      0.94      0.89        17

avg / total       0.77      0.74      0.74       198

[[14  6  0  0  0  0  0  2  0  0  0]
 [ 0 17  0  0  0  0  0  1  0  0  0]
 [ 0  0 20  0  0  0  0  0  0  0  0]
 [ 0  1  0 11  0  2  0  4  0  0  0]
 [ 0  0  0  1  9  1  9  0  0  0  0]
 [ 0  0  0  0  7 12  2  0  0  0  0]
 [ 0  0  0  0  2  2 11  0  0  0  0]
 [ 0  2  0  2  0  0  0 11  0  0  0]
 

In [ ]:
def main_functuion(sampling_rate,*markers):
    x = np.empty([0,3*6*len(markers)])
    y = np.empty([0,])
    #Read all mocap data
    subjects =  ["01","02","03","04","05","06","07","08","09","10","11","12"]
    actions = ["01","02","03","04","05","06","07","08","09","10","11"]
    for subject in subjects:
        for action in actions:
            for record in range(1,6):
                try:
                    data = read_mocap('/Users/takeshin/ws/BerkeleyMHAD/Mocap/OpticalData/moc_s'+str(subject)+'_a'+str(action)+'_r0'+str(record)+'.txt')
                    print('moc_s'+str(subject)+'_a'+str(action)+'_r0'+str(record)+'.txt')
                    columns = [['m'+str(i)+'x','m'+str(i)+'y','m'+str(i)+'z'] for i in markers]
                    cols = np.array(columns)
                    cols = list(np.ravel(cols))
                    cols.append("TimeStamp")
                    data = data[cols]
                    
                    data = fix_NaN(data)
                    i = 0
                    mc_ds = pd.DataFrame()
                    #処理
                    while  i < (len(data)//(480//sampling_rate))*(480//sampling_rate):
                        mc_ds = mc_ds.append(data.iloc[i : i+(480//sampling_rate),:].mean(),ignore_index=True)
                        i+=(480//sampling_rate)
                    if i < len(data):
                        mc_ds = mc_ds.append(data.iloc[i :,:].mean(),ignore_index=True)
                    
                    v = cal_va(mc_ds)
                    times = mc_ds['TimeStamp'].shift(periods=-1)
                    mc_v = pd.DataFrame(v)
                    mc_v["TimeStamp"]=times
                    mc_v = mc_v.dropna()

                    
                    a = cal_va(mc_v)
                    times = mc_v['TimeStamp'].shift(periods=-1)
                    mc_a = pd.DataFrame(a)
                    mc_a["TimeStamp"]=times
                    mc_a = mc_a.dropna()
                
                    
                    #単位を[G]に変換  [mm/s^2] →  [G]
                    accel = mc_a.loc[:, mc_a.columns != 'TimeStamp']*0.001/9.80665
                    accel = accel.rolling(window= 10, center=False).mean()
                    accel = accel.dropna()
                    sf = obtain_statistical_feature(accel)

                    x =np.append(x,sf, axis=0)
                    y = np.append(y,[action], axis=0)
                
                
                except FileNotFoundError:
                    pass
                    
    return x, y

In [ ]:
x,y=main_functuion(30,24,25)   

In [ ]:
x.shape

In [9]:
def ml_ar(X,y):
    x_train, x_test, y_train, y_test = crv.train_test_split(X, y, test_size=0.30, random_state=42)
    clf = RandomForestClassifier(n_estimators=20, random_state=42)
    clf.fit(x_train, y_train)
    y_predict = clf.predict(x_test)
    conf = confusion_matrix(y_test, y_predict)
    print (accuracy_score(y_test, y_predict) ) 
    actions = ["01","02","03","04","05","06","07","08","09","10","11"]
    print(classification_report(y_test, y_predict, target_names=actions))
    print(conf)

In [ ]:
ml_ar(x,y)

In [ ]:
def ml_ar2(X,y):                              
    #Try classifier
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append(('clf', SVC(kernel='linear')))
    model = Pipeline(estimators)
    
    seed = 11
    kf = KFold(n_splits=3, random_state=seed, shuffle=True)
    accuracy = [] 
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train)
        acc_train = model.score(X_train, y_train)
        acc_test = model.score(X_test, y_test)
        accuracy.append(acc_test)
        y_pred = model.predict(X_test)
        conf = confusion_matrix(y_test, y_pred)
        precision = precision_score(y_test, y_pred,average='macro') 
        recall = recall_score(y_test, y_pred,average='macro')
        print("*Accuracy* train:",acc_train, "test:",acc_test)
        #print(conf)
        #print ("precision:",precision,"recall:",recall)
        actions = ["01","02","03","04","05","06","07","08","09","10","11"]
        #actions = ["Jumping in place","Jumping jacks","Bending-hands up all the way down","Punching(boxing)","Waving-two hands","Waving - one hand (right)","Clapping hands","Throwing a ball","09","10","11"]
        print(classification_report(y_test, y_pred, target_names=actions))
        print("-----------")
    print("*test_ave*",sum(accuracy)/len(accuracy))

In [ ]:
ml_ar２(x,y)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import sklearn.cross_validation as crv
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris

x_train, x_test, y_train, y_test = crv.train_test_split(x, y, test_size=0.20, random_state=42)
# 学習
clf = RandomForestClassifier(n_estimators=20, random_state=42)
clf.fit(x_train, y_train)
#予測データ作成
y_predict = clf.predict(x_test)
#正解率
print (accuracy_score(y_test, y_predict) ) 
actions = ["01","02","03","04","05","06","07","08","09","10","11"]
print(classification_report(y_test, y_predict, target_names=actions))

In [ ]:
feature = clf.feature_importances_

#特徴量の重要度を上から順に出力する
f = pd.DataFrame({'number': range(0, len(feature)),
             'feature': feature[:]})
label = [ax+str(i)  for i in range(1,8) for ax in ['x','y','z'] ]
plt.title('Feature Importance')
plt.bar(range(len(feature)),feature, color='lightblue', align='center')
plt.xticks(range(len(feature)),label, rotation=90)
plt.xlim([-1, len(feature)])
plt.tight_layout()
plt.show()